In [1]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import os
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization

Using Theano backend.


# Instead of training models for each note
- decide which will have notes, for now pick x with one, y with another (train model for this + holds, hands, mines, rolls etc later)
- for each note that will have something, decide what combo it has (train from prev notes (not all 48, pick more relevant ones)) (4 for one note, 6 for 2, 4 for 3) and pick highest class

In [2]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.beat_importance = pd.read_csv('generated_data/{0}_importance_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

In [3]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
save_files_generated = listdir('generated_data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_importance_generated.csv'.format(key) in save_files_generated:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])

In [4]:
beats_to_track = 48
num_classes = 3
class_map_one_note = {
    '1000': 0,
    '0100': 1,
    '0010': 2,
    '0001': 3
}

class_map_two_notes = {
    '1001': 0,
    '0110': 1,
    '1100': 2,
    '1010': 3,
    '0101': 4,
    '0011': 5
}

def get_is_note(i, notes):
    if i < 0:
        return [0, 0, 0, 0]
    return [char == '1' for char in notes[i][0]]

def get_is_mine(i, notes):
    if i < 0:
        return [0, 0, 0, 0]
    return [char == 'M' for char in notes[i][0]]

def get_note_class(note):
    if i < 0:
        return 0
    return class_map[note] if note in class_map else 0

def get_row_classes(row):
    return [get_note_class(note) for note in row[0]]

In [14]:
songs['In The Groove~Oasis'].notes[0][0].count('1')

0

In [ ]:
# [12, 24, 36, 48, 6, 18, 4, 8, 1, 2, 3]
X_one_note = []
y_one_note = []
X_two_notes = []
y_two_notes = []
for key in songs:
    beat_importance = songs[key].beat_importance
    notes = songs[key].notes
    length = min(len(beat_importance), len(notes))
    is_note = np.array([get_is_note(j, notes) for j in range(-beats_to_track, length)])
    is_mine = np.array([get_is_note(j, notes) for j in range(-beats_to_track, length)])
    importances = [0 if j < 0 else beat_importance[j][0] for j in range(-beats_to_track, length)]
    for i in range(length):
        steps = notes[i][0].count('1')
        if notes[i][0].count('1')
        
        if get_row_classes(notes[i]) != [0, 0, 0, 0] or random.randint(0, 5) == 0:
            X_row = np.concatenate((is_note[i:i + beats_to_track - 1].flatten(), is_mine[i:i + beats_to_track - 1].flatten(), importances[i:i + beats_to_track]), axis=0)
            X.append(X_row)
            y.append(get_row_classes(notes[i]))

X = np.array(X)
y = np.array(y)

In [5]:
def build_model():
    model = Sequential()

    model.add(Dense(200, input_dim=424, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                               optimizer='adadelta',
                               metrics=['accuracy'])
    
    return model

In [ ]:
#TODO: build models for either side instead of one left one right
def train_models(should_calculate, save_path):
    models = []
    for i in range(4):
        if should_calculate:
            y_transformed = [row[i] for row in y]
            y_one_hot = np.zeros((len(y_transformed), num_classes))
            y_one_hot[np.arange(len(y_transformed)), y_transformed] = 1

            models.append(build_model())
            models[i].fit(np.array(X)[:50000], np.array(y_one_hot)[:50000], nb_epoch=3, batch_size=10)
            models[i].save(save_path.format(i))
        else:
            models.append(load_model(save_path.format(i)))
    return models